<a href="https://colab.research.google.com/github/Flychuban/Pneumonia-Prediction/blob/main/Pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from keras.layers import Dense, Input, Lambda, Flatten
from keras.models import Model, Sequential
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
import keras.utils as image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
from matplotlib import pyplot as plt

In [2]:
# Resize all images
images_size = [224, 224]

train_path = '/content/drive/MyDrive/PneumoniaPrediction/data/train'
test_path = '/content/drive/MyDrive/PneumoniaPrediction/data/test'

In [3]:
vgg = VGG19(input_shape=images_size + [3], weights='imagenet', include_top=False)

80134624/80134624 [==============================] - 1s 0us/step


In [4]:
for layer in vgg.layers:
  layer.trainable = False

In [5]:
folders = glob(train_path + '/*')

In [6]:
folders

['/content/drive/MyDrive/PneumoniaPrediction/data/train/PNEUMONIA',
 '/content/drive/MyDrive/PneumoniaPrediction/data/train/NORMAL']

In [7]:
# Add new layers to the model
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(2, activation='softmax')(x)

In [9]:
# Making new model based on vgg19
model = Model(inputs=vgg.input, outputs=prediction)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [21]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# We are generating more training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode='binary')

Found 5216 images belonging to 2 classes.


In [14]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size=(224, 224),
                                            batch_size=32,
                                            class_mode='binary')

Found 624 images belonging to 2 classes.


In [15]:
history = model.fit(training_set,
                                    validation_data=test_set,
                                    epochs=5,
                                    steps_per_epoch=len(training_set),
                                    validation_steps=len(test_set))

Epoch 1/5
163/163 [==============================] - 595s 3s/step - loss: 0.2224 - accuracy: 0.9070 - val_loss: 0.2570 - val_accuracy: 0.9054
Epoch 2/5
163/163 [==============================] - 140s 856ms/step - loss: 0.1382 - accuracy: 0.9425 - val_loss: 0.3636 - val_accuracy: 0.8702
Epoch 3/5
163/163 [==============================] - 139s 853ms/step - loss: 0.1161 - accuracy: 0.9542 - val_loss: 0.2445 - val_accuracy: 0.9151
Epoch 4/5
163/163 [==============================] - 139s 854ms/step - loss: 0.1112 - accuracy: 0.9559 - val_loss: 0.3572 - val_accuracy: 0.8878
Epoch 5/5
163/163 [==============================] - 140s 858ms/step - loss: 0.1096 - accuracy: 0.9595 - val_loss: 0.3346 - val_accuracy: 0.8942


In [17]:
model.save("pneumonia_model.h5")

AttributeError: ignored

In [20]:
# Test the model
img = image.load_img('/content/drive/MyDrive/PneumoniaPrediction/data/test/PNEUMONIA/person101_bacteria_486.jpeg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)
classes = model.predict(img_data)
classes

1/1 [==============================] - 1s 802ms/step


array([[0., 1.]], dtype=float32)